<a href="https://colab.research.google.com/github/aadityadamle/Random-Forest/blob/main/Random_Forest_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

                                                          REFERENCE

http://mystatisticsblog.blogspot.com/2017/12/building-random-forest-classifier-from.html

https://machinelearningmastery.com/implement-random-forest-scratch-python/





                                           RANDOM FOREST IMPLEMENTATION 


*   Random forest is a machine learning algorithm that produces, a great result most of the time. It is also one of the most used algorithms, because of its simplicity and diversity
* It is a supervised learning algorithm.
* It is an ensemble of decision trees .bagging increases the overall result.
* It can be used for both classification and regression problems.








                                            IMPORTING THE LIBRARIES:

* **randrange** is a function used to call out any random number between the start and stop which are the input features of the function.
* **Pandas** is a famous library that provides various data structures and operations for manipulating numerical data and time series. 



In [ ]:
from random import randrange
import pandas as pd
import math
from math import sqrt


*   load the dataset into a dataframe using pandas.
*   converted the dataframe into list.

                                                    ABOUT DATASET:

* Cardiovascular diseases (CVDs) are the number 1 cause 
of death globally, taking an estimated 17.9 million lives each year, which accounts for 31% of all deaths worlwide.

* Heart failure is a common event caused by CVDs and this dataset contains 12 features that can be used to predict mortality by heart failure.

*   People with cardiovascular disease or who are at high cardiovascular risk (due to the presence of one or more risk factors such as hypertension, diabetes, hyperlipidaemia or already established disease) need early detection and management wherein a machine learning model can be of great help.




    
    





In [ ]:
pip install wget

  Created wheel for wget: filename=wget-3.2-cp36-none-any.whl size=9682 sha256=8cf39f3339051664892aad95e529cf33d43b8cc3706950bb3c23c9a90f617009
  Stored in directory: /root/.cache/pip/wheels/40/15/30/7d8f7cea2902b4db79e3fea550d7d7b85ecb27ef992b618f3f
Successfully built wget


In [ ]:
import wget

In [ ]:
dset = pd.read_csv("/datasets_727551_1263738_heart_failure_clinical_records_dataset.csv")
dset = dset.values.tolist()




*  Defining a function which takes in input: the dataset, index of a column and certain value(value)
*  we iterate through the rows,if the row's value corresponding to the column is greater than (value) we append the row to the right list, elseto the left list.
*   this we we are able to separate the dataset into 2 parts based on the column's value of each row w.r.t the value.








In [ ]:
def testsplit(index, value, dset):
	left, right = list(), list()
	for row in dset:
		if row[index] > value:
			right.append(row)
		else:
			left.append(row)
	return left, right

                                                        CLASS COUNT



*   given a group we define a function which takes input the group and return a list which contains the number of elements under each class.


In [ ]:
def class_counts(group):
    counts = {}  
    for row in group:
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

                                                             ENTROPY

*   entropy is a measure of disorder.
*   entropy is an indicator of how messy the data is.
*   we use entropy as a measure to maximize the purity of the groups as much as possible each time you create a new node of the tree.
*   At each branching, we want to decrease the entropy, so this quantity is computed before the cut and after the cut.
*   more the entropy decreases better the split. 
  






 




In [ ]:
def entropy(group):
    entries = class_counts(group)
    avg_entropy = 0
    size = float(len(group))
    for label in entries:
        prob = entries[label] / size
        avg_entropy = avg_entropy + (prob * math.log(prob, 2))
    return -1*avg_entropy

                                                    INFORMATION GAIN

* Information gain gives us a value which determines how much it is helpful to split a group.higher value indicates that the group should be split 
* It is defined as : 
  
   (entropy of parent group) -  (sum of entropies of child groups)

*   initially we calculate entropy of the whole dataset as save the value in IG.then we iterste over the groups and sum over their individual entropy and substract from the IG calculated initially .
                  

In [ ]:
def Information_gain(groups):
    all_data = [row for group in groups for row in group]
    total_samples = sum([len(group) for group in groups])
    IG = entropy(all_data)
    for group in groups:
        IG -= entropy(group)*len(group)/total_samples
    return IG

                                                         GET SPLIT

                                    

* here we check every value on each attribute as a candidate split, evaluate the cost of the split and find the best possible split we could make.
* This is an exhaustive and greedy algorithm
  
  WORKING:
* a list of features is created and all the index of dataset is appended in it.

*   taking a column into consideration , we check the best split iterating over each row in the column using test split wherein we take the row's index value as threshold value for splitting.

* this whole process is iterated for every column
* IG value for each iteration is recorded.the row of a particular index about which we get highest IG is considered the best split .
* The index ,row[index] value and the groups created are given as output


In [ ]:
def get_split(dset, number_of_features):
  classval = list(set(row[-1] for row in dset))
  split_value, max_IG, split_groups = 0., -1., None 
  features = list()
  while len(features) < number_of_features:
    index = randrange(len(dset[0])-1)
    if index not in features:
       features.append(index)
  for index in features:
    for row in dset:
      groups = testsplit(index, row[index], dset)
      IG = Information_gain(groups)
      if IG > max_IG:
        updated_index, split_value, max_IG, split_groups = index ,row[index], IG, groups
  return {'index': updated_index,'value': split_value,'groups': groups}

        

* **to_terminal** function returns the class with highest number of elements in the group as output.







In [ ]:
def to_terminal(group):
	outcomes = [row[-1] for row in group]
	return max(set(outcomes), key=outcomes.count)

                                                    TERMINATION OF TREE

* we create a function to decide when to stop growing a tree.
* Once a maximum depth of the tree is met, we must stop splitting adding new nodes
* Once at or below minimum node records, we must stop splitting and adding new nodes.
* if we choose a split in which all rows belong to one group then in  this case, we will be unable to continue splitting and adding child nodes.
* splitting of nodes takes place only when none of the above conditions is satisfied.


In [ ]:
def split(node, max_depth, min_size, number_of_features, depth):
	left, right = node['groups']
	del(node['groups'])
 	# check for max depth
	if depth >= max_depth:
		node['left'], node['right'] = to_terminal(left), to_terminal(right)
		return
  # check for a no split
	if not left or not right:
		node['left'] = node['right'] = to_terminal(left + right)
		return
	# process right child
	if len(right) <= min_size:
		node['right'] = to_terminal(right)
	else:
		node['right'] = get_split(right, number_of_features)
		split(node['right'], max_depth, min_size, number_of_features, depth+1)
	 # process left child
	if len(left) <= min_size:
		node['left'] = to_terminal(left)
	else:
		node['left'] = get_split(left, number_of_features)
		split(node['left'], max_depth, min_size, number_of_features, depth+1)

                                                        BUILD TREE

                                     

* using the get split we first get the groups
* then using split function to split nodes as much as possible till full tree is created.



In [ ]:
def build_tree(train, max_depth, min_size, number_of_features):
	root = get_split(train, number_of_features)
	split(root, max_depth, min_size, number_of_features, 1)
	return root

                                                PREDICTION
 We must check if a child node is either a terminal value to be returned as the prediction, or if it is a dictionary node containing another level of the tree to beconsidered. 
 we implement this using the shown  recursive function                                              

In [ ]:
def predict(node, row):
	if row[node['index']] > node['value']:
		if isinstance(node['right'], dict):
			return predict(node['right'], row)
		else:
			return node['right']
	else:
		if isinstance(node['left'], dict):
			return predict(node['left'], row)
		else:
			return node['left']



*   we then create a function to choose the prediction made by all the trees. 
*   this is done by counting the number of same predictions and giving as output the value that has been predicted maximum number of times by the trees.



In [ ]:
def bagging_predict(trees, row):
	predictions = [predict(tree, row) for tree in trees]
	return max(set(predictions), key=predictions.count)

                                                     RANDOM FOREST 



* this is the combination of all the functions made till now.
* we create a list of trees using the train set and predict all their results on the test set which are taken as input parameters



In [ ]:
def random_forest(train, test, max_depth, min_size, n_trees, number_of_features):
	trees = list()
	for i in range(n_trees):
		train_set = list(train)
		tree = build_tree(train_set, max_depth, min_size, number_of_features)
		trees.append(tree)
	predictions = [bagging_predict(trees, row) for row in test]
	return(predictions)



* defining a function to calculate the accuracy of our prediction.
* we compare the actual and predicted for all the prerdictions made and calculate the percentage of correct predictions made.



In [ ]:
def accuracy_metric(actual, predicted):
	correct_prediction = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct_prediction += 1
	return correct_prediction / float(len(actual)) * 100.0


*   devising a function which takes in as input :the dataset ,the algorithm to be applied on the dataset,and the arguments of the algorithm, and returns the accuracy using the previously created functions.




In [ ]:
def algorithm_evaluation(dset, algorithm, n_folds, *args):
  dset_split = list()
  dset_copy = list(dset)
  fold_size = int(len(dset) / n_folds)
  test_set = list()
  while len(test_set) < fold_size:
			index = randrange(len(dset_copy))
			test_set.append(dset_copy.pop(index))
  train_set = list(dset_copy)
  predicted = algorithm(train_set, test_set, *args)
  actual = [row[-1] for row in test_set]
  accuracy = accuracy_metric(actual, predicted)
  return accuracy
 

                                                  TESTING ON DATASET                                 

In [ ]:
n_folds = 5
max_depth = 10
min_size = 1
number_of_features = int(sqrt(len(dset[0])-1))
n_trees = 5

In [ ]:

score= algorithm_evaluation(dset, random_forest, n_folds, max_depth, min_size, n_trees, number_of_features)
score


90.0